In [ ]:
# Real-World Workflow Examples

This notebook contains production-ready workflow examples for common use cases:
- CI/CD pipelines
- Data processing ETL workflows
- Infrastructure automation
- Monitoring and alerting workflows
- Backup and disaster recovery


In [ ]:
import os
from kubiya_workflow_sdk import workflow, step
from kubiya_workflow_sdk.dsl import retry_policy, when
import json

api_key = os.getenv("KUBIYA_API_KEY", "test-key")
print("✅ SDK loaded for real-world examples")


In [ ]:
# Example 1: Complete CI/CD Pipeline
cicd_pipeline = (
    workflow("nodejs-cicd-pipeline")
    .description("Complete CI/CD pipeline for Node.js application")
    .params(
        branch="${GIT_BRANCH:-main}",
        environment="${DEPLOY_ENV:-staging}",
        skip_tests="${SKIP_TESTS:-false}"
    )
    .env(
        NODE_ENV="production",
        NPM_CONFIG_LOGLEVEL="warn"
    )
    .queue("ci-pipeline", max_active_runs=2)
)

# Step 1: Checkout code
checkout_step = step("checkout", "git checkout ${branch} && git pull")
checkout_step.retry(3, 10)
cicd_pipeline.data["steps"].append(checkout_step.to_dict())

# Step 2: Install dependencies with cache
install_step = step("install-deps")
install_step.shell("""
# Check if node_modules cache exists
if [ -f ".npm-cache-hash" ] && [ "$(cat .npm-cache-hash)" = "$(sha256sum package-lock.json)" ]; then
    echo "Using cached dependencies"
    tar -xzf node_modules.tar.gz
else
    echo "Installing dependencies"
    npm ci
    # Cache for next run
    tar -czf node_modules.tar.gz node_modules/
    sha256sum package-lock.json > .npm-cache-hash
fi
""")
cicd_pipeline.data["steps"].append(install_step.to_dict())

# Step 3: Run tests (conditional)
test_step = step("run-tests", "npm test -- --coverage")
test_step.preconditions("${skip_tests} != 'true'")
test_step.output("TEST_RESULT")
test_step.timeout(300)  # 5 minute timeout
cicd_pipeline.data["steps"].append(test_step.to_dict())

# Step 4: Build application
build_step = step("build", "npm run build")
build_step.output("BUILD_VERSION")
cicd_pipeline.data["steps"].append(build_step.to_dict())

# Step 5: Build Docker image
docker_step = step("docker-build")
docker_step.docker(
    image="docker:latest",
    command="docker build -t myapp:${BUILD_VERSION} ."
)
docker_step.depends("build")
cicd_pipeline.data["steps"].append(docker_step.to_dict())

# Step 6: Deploy to environment
deploy_step = step("deploy")
deploy_step.shell("""
case "${environment}" in
    "production")
        kubectl set image deployment/myapp myapp=myapp:${BUILD_VERSION} -n production
        kubectl rollout status deployment/myapp -n production
        ;;
    "staging")
        kubectl set image deployment/myapp myapp=myapp:${BUILD_VERSION} -n staging
        kubectl rollout status deployment/myapp -n staging
        ;;
    *)
        echo "Unknown environment: ${environment}"
        exit 1
        ;;
esac
""")
deploy_step.depends(["docker-build"])
cicd_pipeline.data["steps"].append(deploy_step.to_dict())

# Step 7: Smoke tests
smoke_test_step = step("smoke-tests")
smoke_test_step.shell("""
# Wait for service to be ready
sleep 30

# Run smoke tests
ENDPOINT=$(kubectl get service myapp -n ${environment} -o jsonpath='{.status.loadBalancer.ingress[0].ip}')
curl -f http://${ENDPOINT}/health || exit 1
echo "Smoke tests passed!"
""")
smoke_test_step.retry(3, 20)
smoke_test_step.depends("deploy")
cicd_pipeline.data["steps"].append(smoke_test_step.to_dict())

print("📋 CI/CD Pipeline created:")
print(f"   Steps: {len(cicd_pipeline.data['steps'])}")
print("   Features: caching, conditional tests, rollout verification, smoke tests")
